In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. In this notebook, we use ChatGPT, Gemini on GenerativeAI and Anthropic Claude.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message class allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila import InMemoryConversationMemory, JSONConversationMemory
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: str) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool use

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

In [5]:
from langrila import ToolConfig, ToolParameter, ToolProperty

tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "120", "180"],  # Gemini doesn't support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

### ChatGPT

ToolConfig object is internally converted to client tool configs

In [6]:
chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [7]:
prompt = "Turn this place into a comfortable party mood!"

In [8]:
response = chatgpt.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text="The atmosphere is coming together perfectly! The disco ball is spinning, the energetic music is playing at a lively pace, and the lights have been dimmed to create a cozy vibe. It's time to party! 🎉✨")], name=None), usage=Usage(prompt_tokens=132, completion_tokens=44, total_tokens=176), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_KpwoUNCXBqOjS7secaKPsfdx', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_jgJB0tjz5rkSnuccPfcZaBXL', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_oLNZdpM3YruHzeieBDFl8zUj', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The atmosphere is coming together perfectly! The disco ball is spinning, the energetic music is playing at a lively pace, and the lights have been dimmed to create a cozy vibe. It's time to party! 🎉✨"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 132,
  'completion_tokens': 44},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_KpwoUNCXBqOjS7secaKPsfdx',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_jgJB0tjz5rkSnuccPfcZaBXL',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_oLNZdpM3YruHzeieBDFl8zUj',
     'function'

Asynchronous generation

In [10]:
response = await chatgpt.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is set! The disco ball is spinning, music is playing energetically at 120 BPM, and the lights are dimmed for a cozy atmosphere. Get ready to have a great time! 🎉✨')], name=None), usage=Usage(prompt_tokens=132, completion_tokens=44, total_tokens=176), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a comfortable party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_fRzGz1iVbTJOEqfBmO9bFr62', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_sUlEDjMHX21V7Xz3JFtjOtVu', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_aAUmXsoODzB3r97YOma85MIY', 'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'}, 'type': 'function'}]}, {'role': 'tool', 

multiple generation

In [11]:
response = chatgpt.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The party mood is now set! The lights are dimmed for a cozy atmosphere, the music is pumping with an energetic beat at 120 BPM, and the disco ball is spinning to add some sparkle. Let the fun begin! 🎉'},
  {'text': 'The party mood is set! The lights are dimmed to a comfortable 50%, energetic music is playing at a lively 120 BPM, and the disco ball is spinning to create that perfect atmosphere. Get ready to have fun! 🎉💃🕺'}],
 'name': None}

You can get the results from specific tool.

In [12]:
response = chatgpt.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! The music is pumping with an energetic vibe. Now let's set the mood:\n\n1. **Lighting**: Dim the overhead lights and add some color with LED strips or fairy lights. A disco ball or some fun party lights can enhance the atmosphere.\n\n2. **Seating**: Arrange seating in a cozy, conversational layout. Add some cushions and blankets for comfort.\n\n3. **Beverages**: Set up a drink station with a mix of alcoholic and non-alcoholic options. Don’t forget some fun party cups!\n\n4. **Snacks**: Prepare a variety of finger foods and snacks. Think chips, dips, and a grazing platter with fruits and cheese.\n\n5. **Games**: Set up some fun party games or activities to keep everyone entertained, like a photo booth with props or classic board games.\n\n6. **Themed Decor**: If there's a specific theme, add decorations that reflect it to bring some extra flair!\n\nNow let's get the laughter and conversation flowing! Enjoy the party! 🎉"}],

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [13]:
response = chatgpt.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=169, completion_tokens=75, total_tokens=244), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_fhokT12rQFvHhrHmhvsKErLS', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_FDOCJVXZ3UGa2ADnBem51bvo', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_2nxYQT8E9EmhyRgPqVWRDyKL', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_fhokT12rQFvHhrHmhvsKErLS'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_F

In [14]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 169,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_fhokT12rQFvHhrHmhvsKErLS',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
     'call_id': 'call_FDOCJVXZ3UGa2ADnBem51bvo',
     'args': '{"energetic": true, "loud": true, "bpm": "120"}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_2nxYQT8E9EmhyRgPqVWRDyKL',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_fhokT12rQFvHhrHmhvsKErLS'}

In [15]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_fhokT12rQFvHhrHmhvsKErLS', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output="Starting music! energetic=True loud=True, bpm='120'", call_id='call_FDOCJVXZ3UGa2ADnBem51bvo', args='{"energetic": true, "loud": true, "bpm": "120"}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_2nxYQT8E9EmhyRgPqVWRDyKL', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [16]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_fhokT12rQFvHhrHmhvsKErLS'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": "120"}', call_id='call_FDOCJVXZ3UGa2ADnBem51bvo'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_2nxYQT8E9EmhyRgPqVWRDyKL')], name=None)

### Gemini on Google Generative AI

You can use tools for Gemini as well which defined for ChatGPT.

In [17]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [18]:
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, the music pumping, and the lights dimmed! Let's get this party started! 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 29},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      

Asynchronous generation

In [19]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've dimmed the lights, turned on the disco ball, and started some energetic music.  Let's get this party started!  What else can I do to set the mood? 🎉 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 42},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "dim_lights"
      args {
        fields {
          key: "brightness"
          value {
            number_value: 0.5
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
    

Gemini allows you to call specific tool as well as ChatGPT.

In [20]:
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm setting the mood!  Let's get this party started! 🎉 I'm putting on some energetic music with a good beat.  What else can I do to make it feel festive in here?  🥳 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 68,
  'completion_tokens': 48},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"


### Gemini on VertexAI

In [21]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [22]:
response = gemini_vertexai.run(prompt=prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Okay, I'm getting this place ready for a party!\n\n* **Music's on!** I've got some upbeat tunes playing with a good tempo (around 120 bpm).  It's not too loud, so people can still chat.\n* **Low lighting:** The lights are dimmed to a nice, cozy level. \n* **Disco ball time!**  The disco ball is spinning, adding a bit of fun sparkle.  \n\nNow we're getting somewhere!  What else can I do to get the party started? \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-flash',
  'prompt_tokens': 73,
  'completion_tokens': 119},
 'prompt': [role: "user"
  parts {
    text: "Turn this place into a comfortable party mood!"
  },
  role: "model"
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: false
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }


multiple generation

In [23]:
response = gemini_vertexai.run(prompt=prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Alright, let's get this party started! I've got some groovy tunes playing, dimmed the lights for a cozy ambiance, and even turned on the disco ball.  Let's get this party poppin'!  💃🕺🎉 \n\nWhat else can I do to help you create the perfect party mood? Maybe you want to adjust the music, set the temperature, or even order some snacks?  Just let me know! \n"},
  {'text': "Alright, let's get this party started! I've got some groovy tunes playing, dimmed the lights for a cozy ambiance, and even turned on the disco ball.  Let's get this party poppin'!  💃🕺🎉 \n\nWhat else can I do to help you create the perfect party mood? Maybe you want to adjust the music, set the temperature, or even order some snacks?  Just let me know! \n"}],
 'name': None}

### Anthropic Claude

tool_configs can be shared with Claude.

In [24]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [25]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights: I've set the brightness to 0.4, which creates a cozy and inviting atmosphere without being too dark.\n\n2. Started the music: I've chosen energetic music to keep the party mood up, but set it to not be too loud so people can still comfortably chat. The beats per minute (BPM) is set to 120, which is a good mid-tempo range for a comfortable party vibe.\n\n3. Activated the disco ball: The spinning disco ball adds a fun, party-like visual element to the room without being overwhelming.\n\nThese settings should create a welcoming and enjoyable atmosphere for your party. The dimmed lights and spinning disco ball will create a nice ambiance, while the energetic but not too loud music will keep the mood upbeat while still allowing for conversation.\n\nIs there anything else you'd like me to adjust to perfect the party mood?"}],
  'name':

Asynchronous generation

In [26]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy and relaxed atmosphere without being too dark.\n2. Started playing some music that's not too energetic or loud, with a moderate tempo of 120 beats per minute. This should provide a nice background ambiance without overwhelming conversation.\n3. Activated the disco ball to add a fun, festive touch to the room without being too intense.\n\nThis combination should create a comfortable party mood that's perfect for socializing and enjoying the atmosphere. The dimmed lights and gentle music will make it easy for people to chat and relax, while the disco ball adds a touch of fun and celebration.\n\nIs there anything else you'd like to adjust to make the party mood even more comfortable or suited to your preferences?"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240

### Claude on Amazon Bedrock

In [27]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
)

In [28]:
response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up a comfortable party mood for you. Here's what I've done:\n\n1. Dimmed the lights to 40% brightness (0.4), which should create a cozy, party-like atmosphere without being too dark.\n2. Started some energetic music at 120 BPM (beats per minute), which is a medium tempo good for casual dancing and socializing. I've set it to not be too loud so people can still converse comfortably.\n3. Powered on the disco ball to add some fun, moving lights to the space.\n\nThis combination should create a comfortable party mood that's lively enough for fun but still allows for easy conversation. The dimmed lights, upbeat music, and spinning disco ball all contribute to a festive atmosphere.\n\nIs there anything else you'd like to adjust? Perhaps you'd like the music louder or softer, or the lights brighter or dimmer? Feel free to let me know if you want any changes!"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonn

# Universal tool config system

Tool config the user defines is common for various llm. It is converted to client tools in clinet's FunctionCallingModule before API call. Let me show you the process of conversion.

In [29]:
from langrila.openai import OpenAIToolConfig

converted_configs = OpenAIToolConfig.from_universal_configs(tool_configs)
converted_configs

[OpenAIToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None)], required=['power']), type='function'),
 OpenAIToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None), OpenAIToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None), OpenAIToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'])], required=['energetic', 'loud', 'bpm']), type='function'),
 OpenAIToolConfig(name='dim_lights', description='Dim the lights.', parameters=OpenAIToolParameter(type='object', properties=[OpenAIToo

Then these client tool config objects is transformed to client tools by format() method.

In [30]:
converted_configs[0].format()

{'type': 'function',
 'function': {'name': 'power_disco_ball',
  'description': 'Powers the spinning disco ball.',
  'parameters': {'type': 'object',
   'properties': {'power': {'type': 'boolean',
     'description': 'Boolean to spin disco ball.'}},
   'required': ['power']}}}

Other client like Gemini and Claude is the same interface, so you could reuse tool_configs you initially defines.

In [31]:
from langrila.gemini.genai import GeminiToolConfig

In [32]:
converted_configs = GeminiToolConfig.from_universal_configs(tool_configs)
converted_configs

[GeminiToolConfig(name='power_disco_ball', description='Powers the spinning disco ball.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='power', type='boolean', description='Boolean to spin disco ball.', enum=None)], required=['power'])),
 GeminiToolConfig(name='start_music', description='Play some music matching the specified parameters.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='energetic', type='boolean', description='Whether the music is energetic or not.', enum=None), GeminiToolProperty(name='loud', type='boolean', description='Whether the music is loud or not.', enum=None), GeminiToolProperty(name='bpm', type='string', description='The beats per minute of the music.', enum=['60', '120', '180'])], required=['energetic', 'loud', 'bpm'])),
 GeminiToolConfig(name='dim_lights', description='Dim the lights.', parameters=GeminiToolParameter(type='object', properties=[GeminiToolProperty(name='brightness', type=

In [33]:
converted_configs[0].format()

name: "power_disco_ball"
description: "Powers the spinning disco ball."
parameters {
  type_: OBJECT
  properties {
    key: "power"
    value {
      type_: BOOLEAN
      description: "Boolean to spin disco ball."
    }
  }
  required: "power"
}

# Multi-turn conversation with tools

### For ChatGPT

In [34]:
chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [35]:
prompt = "Turn this place into a comfortable party mood!"

response = chatgpt.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music with a lively BPM of 120 is pumping through the air, and the lights are dimmed to create the perfect ambiance. Time to have fun! 🎉✨'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 132,
  'completion_tokens': 47},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_Z1iKn8YoQ5aXQq2UxTLEGQ87',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_65xrAkbRI5yBjxoIETVFdK7Z',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_dI44ZPP5v4jclH5bJPzTDEw4',
     'function':

Next turn

In [36]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The BPM has been boosted to 180! Get ready to dance and enjoy the high-energy vibes! 🎶💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 240,
  'completion_tokens': 27},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_Z1iKn8YoQ5aXQq2UxTLEGQ87',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_65xrAkbRI5yBjxoIETVFdK7Z',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_dI44ZPP5v4jclH5bJPzTDEw4',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'function'}]},
  {'role': 'tool',
  

### For Gemini

In [37]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [38]:
prompt = "Turn this place into a comfortable party mood!"

response = gemini.run(prompt=prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've got the disco ball spinning, the music pumping, and the lights dimmed!  Let's get this party started! 🎉 🎶 🕺 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 145,
  'completion_tokens': 34},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
 

In [39]:
prompt = "What is the current bpm?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The current bpm is 120.  Get ready to move! 🎶 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 189,
  'completion_tokens': 17},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "dim_lights"
    

In [40]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, I've cranked up the tempo! The bpm is now 180! Time to really get this party going! 🚀🥳 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 275,
  'completion_tokens': 30},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  parts {

## Multi-turn conversation using tools with multiple model

In [41]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=shared_memory,
)

In [42]:
prompt = "Turn this place into a comfortable party mood!"

response = chatgpt.run(prompt)

In [43]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! 🎶\n\n- **Energetic music** is playing at a lively 120 BPM!\n- The **disco ball** is spinning, adding that perfect sparkle!\n- The **lights are dimmed** to create an inviting ambiance.\n\nGet ready to have a great time! 🎉'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 132,
  'completion_tokens': 64},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Turn this place into a comfortable party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_ns41WV7V7FLsuLVscWQgSNXY',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_nzcZpTnzcZdVv6z5VPqwL5qA',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'cal

Gemini can continue conversation without any additional process.

In [44]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, things just got even more intense! I've bumped the BPM up to a thumping 180. Get ready to dance! 💃🕺 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 279,
  'completion_tokens': 33},
 'prompt': [parts {
    text: "Turn this place into a comfortable party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "120"
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }


Gemini also works as well.

NOTE : Claude may not work in multi-turn conversation using tools. Bug fix is in progress.

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [45]:
from langrila import InMemoryConversationMemory

chatgpt = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [46]:
prompt = "Turn this place into a comfortable party mood!"

response = chatgpt.run(prompt, tool_choice="auto")

In [47]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [48]:
print(response.message.content[0].text)

The music is now pumping at 180 BPM! Get ready to dance like never before! 🎶💃✨


In [49]:
response.usage

Usage(prompt_tokens=209, completion_tokens=23, total_tokens=232)

In [50]:
response.prompt

[{'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_B9VHWRpJwRxMvCtiMf7P1aQD',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_OofF1HiwZTy50phlsdV5DTYA',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": "120"}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_PjPY9SvS0LSK0VUsCRKtudR1',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_B9VHWRpJwRxMvCtiMf7P1aQD',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_OofF1HiwZTy50phlsdV5DTYA',
  'name': 'start_music',
  'content': "Starting music! energetic=True loud=True, bpm='120'"},
 {'role': 'tool',
  'tool_call_id': 'call_PjPY9SvS0LSK0VUsCRKtudR1',
  'name': 'dim_lights',
  'content': 'Lights are now set to 0.5'},
 {'role': 

In [51]:
# whole history
chatgpt.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a comfortable party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_B9VHWRpJwRxMvCtiMf7P1aQD'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'call_id': 'call_OofF1HiwZTy50phlsdV5DTYA'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_PjPY9SvS0LSK0VUsCRKtudR1'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_B9VHWRpJwRxMvCtiMf7P1aQD',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': "Starting music! energetic=True loud=True, bpm='120'",
    'call_id': 'call_OofF1HiwZTy50phlsdV5DTYA',
    'args': '{"energetic": true, "loud": true, "bpm": "120"}',
    'funcname': 'start_music'}],
  'name': None},
 {